In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import r2_score

from sklearn.gaussian_process import GaussianProcessRegressor 
from sklearn.gaussian_process import kernels  


from bayes_opt import BayesianOptimization
from bayes_opt import acquisition

from pyfitit import descriptor
from pyfitit import ML
import matplotlib as mpl
import matplotlib.colors as mcolors
import os
import copy

In [2]:
def cv(x, y, model):
    pred = []
    loo = LeaveOneOut()
    if type(x) is not np.array:x =np.array(x)
    if type(y) is not np.array:y = np.array(y)
    for i, (train_index, test_index) in enumerate(loo.split(x)):
        model.fit(x[train_index], y[train_index])
        res=model.predict(np.array(x[test_index]))
        pred.append(float(res))
    return r2_score(y, pred)

In [3]:
def custom_cmap(c1,c2,n):
    """Return object cmap.
        :c1: array [r,g,b] 0-1 or 0-255. or color name from matplotlib
        :c2: array [r,g,b] 0-1 or 0-255. or color name from matplotlib
        :n: number of gradation
    """
    if type(c1) != str:
        c1=np.array(c1)
        c2=np.array(c2)
        if np.logical_or(c1>1, c2>1).any():
            c1=c1/255
            c2=c2/255
    else:
        c1=mpl.colors.to_rgb(c1)
        c2=mpl.colors.to_rgb(c2)
    q=np.array([np.linspace(c1[0] ,c2[0],n),np.linspace(c1[1] ,c2[1],n),np.linspace(c1[2] ,c2[2],n),[1]*n])
    colorMap=mpl.colors.ListedColormap(q.T)
    return colorMap

def norm(mas):
    if len(mas.shape)==1:
        return (mas-np.amin(mas))/(np.amax(mas)-np.amin(mas))
    else:
        for i in range(len(mas)):
            mas[i]=(mas[i]-np.amin(mas[i]))/(np.amax(mas[i])-np.amin(mas[i]))
    return mas

def norm_min_max(mas,min=None,max=None):
    if type(min) is float or type(min) is int:
        if len(mas.shape)==1:
            return (mas-min)/(max-min)
        else:
            for i in range(len(mas)):
                mas[i]=(mas[i]-min)/(max-min)
    else:
        for i in range(len(mas)):
            mas[i]=(mas[i]-min[i])/(max[i]-min[i])
    return mas

def gradient(x, y):
    q,t=[],[]
    for s in np.argsort(x):
        q.append(x[s])
        t.append(y[s])
    grad= []
    for i in range(len(q)-1):
        if q[i+1]-q[i]==0: 
            grad.append(0)
        else:
            grad.append((t[i+1]-t[i])/(q[i+1]-q[i]))
    return np.max(np.abs(grad))


def ccmap(countr):
    lev=countr.levels
    cmap=mpl.colormaps.get_cmap('viridis')
    cmap=cmap(np.linspace(0,1,len(lev)+1))
    k=0
    for k in range(len(lev)):
        if lev[k]<=9.4: 
            print(k, lev[k])
            cmap[k]=[0.5,0.5,0.5,1]
        else:
            lev[k-1]=9.4
            break
    cmap=mcolors.ListedColormap(cmap)
    return lev,cmap

def denorm(mas,min,max):
    return mas*(max-min)+min

def posterior(optimizer, grid):
    mu, sigma = optimizer._gp.predict(grid, return_std=True)
    return mu, sigma 

In [42]:
def plotter(
    optimizer,
    acquisition_f,
    bounds,
    x,
    y,
    param=[],
    model=None,
    new_point=None,
    name="",
    folder_name='maps_optimizator'
):
    os.makedirs(folder_name, exist_ok=True)
    
    fig, axs = plt.subplots(1,2, figsize=(18, 8), dpi=300)
    fig.suptitle(name)
    resolution = 100
    format = "%.2f"
    
    
    delta0 = (bounds[0][1] - bounds[0][0]) / 10
    delta1 = (bounds[1][1] - bounds[1][0]) / 10

    aa, bb = np.meshgrid(
        np.linspace(bounds[0][0] - delta0, bounds[0][1] + delta0, resolution),
        np.linspace(bounds[1][0] - delta1, bounds[1][1] + delta1, resolution),
    )

    xx=np.vstack([aa.ravel(),bb.ravel()]).T

    
    acquisition_f._fit_gp(optimizer._gp, optimizer._space)
    qq = acquisition_f._get_acq(gp=optimizer._gp)(xx)
    utility = -qq.reshape(aa.shape)
    _model=model
    _model.fit(x, y)
    z = _model.predict(xx).reshape(resolution, resolution)

    mu, sigma = posterior(optimizer, xx)
    # mu = mu + 1.9600 * sigma #у среднее предсказанное + доерительный интервал
    mu = mu.reshape(resolution, resolution)
    sigma = sigma.reshape(resolution, resolution)

    plotparam={'levels':10, 'cmap':'viridis','aspect_ratio':1}
    plotparam2={'levels':10,'colors':'black','linewidths':0.5}

    # axs[0, 0].set_title("mu (predict value)")
    # countr = axs[0, 0].contourf(aa, bb, mu,  **plotparam)
    # axs[0, 0].contour(aa, bb, mu,**plotparam2)
    # fig.colorbar(countr, ax=axs[0,0], format=format)

    # axs[0, 1].set_title("sigma")
    # countr = axs[0, 1].contourf(aa, bb, sigma, **plotparam)
    # axs[0, 1].contour(aa, bb, sigma,**plotparam2)
    # fig.colorbar(countr, ax=axs[0,1], format=format)

    axs[0].set_title("model, r2:" + str(round(cv(x, y, _model), 2)))
    countr = axs[0].contourf( aa, bb, z ,**plotparam)
    axs[0].contour( aa, bb, z ,**plotparam2)
    fig.colorbar(countr, ax=axs[0], format=format)

    axs[1].set_title("utility func")
    countr = axs[1].contourf(aa, bb, utility,**plotparam)
    axs[1].contour(aa, bb, utility,**plotparam2)
    fig.colorbar(countr, ax=axs[1], format=format)


    i=0
    for j in range(2):
        axs[j].set_aspect = 1
        
        axs[j].set_xlabel(param[0])
        axs[j].set_ylabel(param[1])
        axs[j].scatter(
            x.T[0], x.T[1], color="red", marker="o", alpha=0.5, label="real point"
        )
        if new_point is not None:
            new_point = np.array(new_point)
            axs[j].scatter(
                new_point[len(new_point) - 1][0],
                new_point[len(new_point) - 1][1],
                color="black",
                alpha=0.5,
            )
            axs[j].scatter(
                new_point[: len(new_point) - 1, 0],
                new_point[: len(new_point) - 1, 1],
                color="blue",
                alpha=0.5,
            )

    fig.tight_layout()

    fig.savefig(folder_name + "/" + str(name) + ".png")
    plt.cla()
    plt.clf()
    plt.close()

In [38]:
def batchBayes(
    data,
    param=[],
    target="",
    name="",
    bounds=[],
    num_suggest_points=1,
    rounded=4,
    model_map=None,
    model_opt=None,
    acquisition_function=None,
    optimizator_parameters={},
    plot_history=False,
    plot_optimization=False,
    plotDescriptors2d=False,
    do_cv=True,
    folder='maps_optimizator'
):
    if plot_history and len(param)!=2: plot_history=False
    if plot_optimization and len(param)!=2: plot_optimization=False
    if plotDescriptors2d and len(param)!=2: plotDescriptors2d=False


    #просто model -объект для передачи в другие функции, а _model для использования здесь
    if model_map is None:
        model_map = copy.deepcopy(ExtraTreesRegressor(random_state=0))
    if model_map =='same':
        from sklearn.gaussian_process.kernels import Matern
        model_map = GaussianProcessRegressor(
                    kernel=Matern(nu=2.5),
                    alpha=1e-6,
                    normalize_y=True,
                    n_restarts_optimizer=5,
                    random_state=0,
                )
    else:
        model_map=copy.deepcopy(model_map)


    x = pd.DataFrame(data, columns=param).to_numpy()
    y = pd.DataFrame(data, columns=[target]).to_numpy().ravel()
    if do_cv:
        print("r2 score: ", cv(x, y, model_map))  # результат кросс-валидации

    model_map.fit(x, y)  # финальное обучение модели

    # создание оптимизатора

    if len(bounds)==0:
        bounds=[]
        for p in param:
            _a=[data[p].min(), data[p].max()]
            bounds.append(_a)
            print('bounds ',p,':',_a)

    pbounds = dict(zip(param, bounds))

    if acquisition_function is None:
        acquisition_function = acquisition.UpperConfidenceBound(random_state=0)


    _def_optim_parametrs= {'f':None,'pbounds':pbounds,'acquisition_function':acquisition_function,'verbose':0,'constraint':None,'random_state':0,'allow_duplicate_points':False}

    if len(optimizator_parameters) >0:
        _def_optim_parametrs.update(optimizator_parameters)
    optimizator_parameters=_def_optim_parametrs

    optimizer = BayesianOptimization(**optimizator_parameters)

    # добавление данных  в его историю
    for i in range(len(data)):
        optimizer.register(params=dict(zip(param, x[i])), target=y[i])
        if plot_history==True:
            plotter(
                copy.deepcopy(optimizer),
                copy.deepcopy(acquisition_function),
                bounds,
                x[: i + 1],
                y[: i + 1],
                param=param,
                model=copy.deepcopy(model_map),
                name="history " +target+' '+ str(i + 1),
                folder_name=folder
            )
        if plot_history=='last' and i>len(data)-10:
            plotter(
                copy.deepcopy(optimizer),
                copy.deepcopy(acquisition_function),
                bounds,
                x[: i + 1],
                y[: i + 1],
                param=param,
                model=copy.deepcopy(model_map),
                name="history " +target+' '+ str(i + 1),
                folder_name=folder
            )

    # генерация точки, предсказание ее значения и добавления в датафрейм
    points = []
    sp_ = []
    for i in range(num_suggest_points):
        next = optimizer.suggest()  # предлагаемая точка
        sp = [] # получение ее значение, иначе он, гад такой, сортирует параметры по имени
        for pname in param:
            sp.append(round(next.get(pname), rounded))
        sp_.append(sp)

        res = float(model_map.predict(np.array([sp])))  # предсказание ответа в точке

        points.append(["sp " + str(i + 1)] + sp + [res])  # сохранение точки и ответа в массив

        if plot_optimization==True:
            plotter(
                copy.deepcopy(optimizer),
                copy.deepcopy(acquisition_function),
                bounds,
                x[: len(data)],
                y[: len(data)],
                param=param,
                model=copy.deepcopy(model_map),
                new_point=sp_,
                name="optimization "+target+' '+ str(i + 1),
                folder_name=folder
            )
        
        optimizer.register(params=next, target=res)  # запись данных в историю оптимизатора

    new_points = pd.DataFrame(points, columns=[name, *param, target])

    if plotDescriptors2d:
        descriptor.plotDescriptors2d(
            data,
            descriptorNames=param,
            labelNames=[target],
            cv_count=len(data),
            folder_prefix="maps_plotDescriptors2d/",
            unknown=new_points,
            dpi=200,
            cmap="plasma",
            plotPadding=0.1,
            model_regr=model_map,
            # use_custom_cmap=True,
            save_cv=False,
            add_len_name=True,
            textColumn=name,
            last_black_point=1,
        )

    return new_points

In [39]:
data = pd.read_excel("журнал.xlsx")

In [40]:
data.head()

,name,"tBuAcr, m%","StAcr, m%",FtD,"Link, m%","Abs, m%","Sens, m%",Контактный угол,Механика,умножение
0,1.1,85.315556,10.664444,0,3.0,0.02,1,98.2,3.0,294.60
1,1.2,84.426667,10.553333,0,4.0,0.02,1,97.9,3.5,342.65
2,1.3,83.537778,10.442222,0,5.0,0.02,1,97.5,4.0,390.00
3,1.4,82.648889,10.331111,0,6.0,0.02,1,96.9,5.5,532.95
4,1.5,81.760000,10.220000,0,7.0,0.02,1,96.5,6.0,579.00


In [44]:
for target in ['Контактный угол','Механика','умножение']:
        batchBayes(
        data,
        param=["StAcr, m%",'Link, m%'],
        target=target,
        name="name",
        model_map='same',
                # y1,y2,x1,x2
        # bounds=[[0,30],[0,30]],
        # acquisition_function= acquisition.UpperConfidenceBound(random_state=0),
        optimizator_parameters={'allow_duplicate_points':True},
        plot_history='last',
        plot_optimization=True,
        # plotDescriptors2d=True,
        num_suggest_points=5,
        do_cv=True,
        rounded=4,
        folder=target
        )

r2 score:  -0.17443149688844706
bounds  StAcr, m% : [3.0659, 25.714]
bounds  Link, m% : [1.978, 24.626099999999997]
r2 score:  0.6681564908209274
bounds  StAcr, m% : [3.0659, 25.714]
bounds  Link, m% : [1.978, 24.626099999999997]
r2 score:  0.5939931058916494
bounds  StAcr, m% : [3.0659, 25.714]
bounds  Link, m% : [1.978, 24.626099999999997]
